# ⚙️ Module 13: ADAS Implementation

This is the implementation phase. You will combine all the theory and individual components from the previous modules into one complete, functional Advanced Driver Assistance System: a Forward Collision Warning (FCW) system.

## Section 1: Your Mission

Your goal is to build a **complete Forward Collision Warning system** that:
1. Reads live motor speed (RPM) from the VESC.
2. Reads live object detections from the AI camera.
3. Estimates the distance to a detected 'person'.
4. Calculates the Time-to-Collision (TTC).
5. Issues multi-level audio alerts based on the TTC value.
6. Displays a real-time status dashboard.

## Section 2: Component Integration

First, let's bring all our functions and initializations into one place.

In [ ]:
# 2.1 - Full System Initialization
import sys, os, time
from gpiozero import Buzzer
from IPython.display import display, HTML, clear_output
sys.path.append(os.path.abspath('../../'))
from student_api import VESCStudentAPI, AIStudentAPI

# -- Initialize Hardware --
buzzer = Buzzer(17)
ai_api = AIStudentAPI()
ai_api.start_camera()
vesc_api = VESCStudentAPI()
vesc = None
if vesc_api.start():
    time.sleep(2) # Short wait for discovery
    controllers = vesc_api.get_connected_controllers()
    if controllers:
        vesc = vesc_api.get_controller(controllers[0])
        print(f'✅ VESC & AI Systems Initialized.')
    else:
        print('❌ VESC: No controllers found')
else:
    print('❌ VESC: Failed to start API')

In [ ]:
# 2.2 - Helper Functions from Module 12

# IMPORTANT: These are simplified placeholder values. 
# A real system would require careful calibration!
WHEEL_CIRCUMFERENCE_M = 0.2 # 20cm wheel
GEAR_RATIO = 12.0 # 12 motor rotations per 1 wheel rotation
CALIBRATION_DISTANCE_M = 2.0
CALIBRATION_BOX_HEIGHT_PX = 150 

def estimate_distance(box_height_px):
    if box_height_px == 0:
        return float('inf')
    return (CALIBRATION_BOX_HEIGHT_PX / box_height_px) * CALIBRATION_DISTANCE_M

def get_speed_mps(rpm):
    # Convert RPM to meters per second
    revolutions_per_second = rpm / 60.0
    wheel_rps = revolutions_per_second / GEAR_RATIO
    speed_mps = wheel_rps * WHEEL_CIRCUMFERENCE_M
    return speed_mps

def calculate_ttc(distance_m, speed_mps):
    if speed_mps <= 0.1: # Threshold to avoid division by zero or tiny speeds
        return float('inf')
    return distance_m / speed_mps

print("✅ Helper functions loaded.")

## Section 3: Complete FCW System

Now we combine everything into a single, continuous loop. Press the 'Interrupt' (square) button to stop the system.

In [ ]:
# --- FCW Configuration ---
CONFIDENCE_THRESHOLD = 0.7
TARGET_LABEL = 'person'
COOLDOWN = 3 # seconds
last_alert_time = 0

# --- Alert Thresholds (in seconds) ---
TTC_CRITICAL = 1.0
TTC_WARNING = 3.0

try:
    print("✅ Forward Collision Warning System ACTIVE. Press Interrupt (■) to stop.")
    while True:
        # 1. GET DATA
        rpm = vesc.get_rpm() if vesc else 0
        _, detections = ai_api.get_frame_and_detections()
        
        # 2. PROCESS DATA
        speed_mps = get_speed_mps(rpm)
        closest_person_dist = float('inf')

        for det in detections:
            if det['label'] == TARGET_LABEL and det['confidence'] > CONFIDENCE_THRESHOLD:
                box_height = det['box'][3]
                dist = estimate_distance(box_height)
                if dist < closest_person_dist:
                    closest_person_dist = dist
        
        ttc = calculate_ttc(closest_person_dist, speed_mps)
        
        # 3. DETERMINE ALERT LEVEL
        status = "SAFE"
        alert_level = 0
        if ttc < TTC_CRITICAL:
            status = "CRITICAL"
            alert_level = 2
        elif ttc < TTC_WARNING:
            status = "WARNING"
            alert_level = 1
            
        # 4. HANDLE ALERTS & COOLDOWN
        if alert_level > 0 and (time.time() - last_alert_time > COOLDOWN):
            last_alert_time = time.time()
            if alert_level == 2: # Critical
                buzzer.beep(0.1, 0.1, 3)
            elif alert_level == 1: # Warning
                buzzer.beep(0.3, 0.5, 1)

        # 5. DISPLAY DASHBOARD
        clear_output(wait=True)
        status_color = 'lime' if status == 'SAFE' else ('orange' if status == 'WARNING' else 'red')
        html_out = f"""
        <div style="border: 2px solid #44B6E5; padding: 10px; font-family: 'Space Grotesk', monospace;">
             <p style="font-size: 24px;">FCW STATUS: <strong style="color:{status_color};">{status}</strong></p>
             <p><strong>Time-to-Collision:</strong> {ttc:.2f} s</p>
             <p><strong>Est. Distance:</strong> {closest_person_dist:.2f} m</p>
             <p><strong>Speed:</strong> {speed_mps * 3.6:.1f} km/h</p>
        </div>
        """
        display(HTML(html_out))
        
        time.sleep(0.1)

except KeyboardInterrupt:
    print("\nFCW System stopped by user.")
finally:
    # Clean up hardware connections
    ai_api.stop_camera()
    if vesc_api.is_running(): vesc_api.stop()
    buzzer.close()

## Section 4: Testing

**SAFETY FIRST: Ensure the vehicle is on a secure test stand with the wheels OFF THE GROUND.**

### EXERCISE: System Validation
Follow this protocol to test your system. Document your observations.

1. **Baseline (No Person):** Run the system and spin the motor wheel by hand. 
   - *Expected:* The dashboard should show changing speed and TTC='inf', but the status should remain 'SAFE' and the buzzer should be silent. 
   - *Observed:*

2. **Stationary Test (Person Detected):** Run the system. Have a team member stand in front of the camera, but DO NOT spin the wheel.
   - *Expected:* The dashboard should show a distance and TTC='inf'. The status should remain 'SAFE' and the buzzer should be silent.
   - *Observed:*

3. **Dynamic Test (Person + Motion):** Run the system. Have a team member stand in front of the camera, and then gently spin the motor wheel by hand.
   - *Expected:* As the speed increases, the TTC should decrease, and the status should change from SAFE -> WARNING -> CRITICAL, with corresponding beeps.
   - *Observed:*

## Section 5: Knowledge Check

### EXERCISE: List 3 Improvements
Based on your testing, what are three ways you could make this FCW system more reliable or intelligent? (e.g., consider object size, false positives, etc.)

// 1. 
// 2. 
// 3. 

### EXERCISE: What's Next?
Our system only *warns* the driver. What is the next logical step for a more advanced system like Automatic Emergency Braking (AEB)? What VESC command from Module 4 would be essential for this?

// Your answer here

---
## Congratulations! You've completed Module 13 and built a real ADAS feature.